In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker # Para formatação de gráficos
from io import StringIO # Para ler CSV de string, caso necessário

# --- 0. Carregamento e Preparação Inicial do Dataset 1 ---
nome_arquivo_dataset1 = './data/energy30days.csv'

try:
    energy_daily_raw = pd.read_csv(nome_arquivo_dataset1)
    print(f"Dataset '{nome_arquivo_dataset1}' carregado com sucesso.")
except FileNotFoundError:
    print(f"\nERRO: O arquivo '{nome_arquivo_dataset1}' não foi encontrado.")
    print("Por favor, verifique o nome do arquivo e se ele está na mesma pasta do seu script/notebook.")
    # Exemplo de como usar StringIO se o erro persistir após upload:
    # csv_data = """Date and time,SMV340200 - Total energy consumed (Wh, Sum over last 1 day) ()...
    # (cole o conteúdo completo do seu CSV aqui)
    # """
    # energy_daily_raw = pd.read_csv(StringIO(csv_data))
    exit() # Interrompe a execução se o arquivo não for encontrado
except Exception as e:
    print(f"\nOcorreu um erro ao carregar ou processar o arquivo: {e}")
    exit()

# 1. Renomear a coluna de data/hora para 'timestamp' e converter para datetime
energy_daily_raw.rename(columns={"Date and time": "timestamp"}, inplace=True)
energy_daily_raw['timestamp'] = pd.to_datetime(energy_daily_raw['timestamp'])

# 2. Extrair os IDs dos equipamentos dos nomes das colunas
# As colunas de equipamentos começam da segunda coluna em diante (índice 1)
equipment_cols_full_names = energy_daily_raw.columns[1:]
# Usamos .split(" - Total energy consumed")[0] para pegar apenas o ID do equipamento
equipment_ids = [col.split(" - Total energy consumed")[0] for col in equipment_cols_full_names]

# Renomear colunas do DataFrame para os IDs limpos
energy_daily_raw.columns = ['timestamp'] + equipment_ids 

# 3. Tratar valores ausentes (strings vazias ou outros não numéricos) e converter para numérico
# errors='coerce' converterá valores não numéricos (como strings vazias "") para NaN
for col in equipment_ids:
    energy_daily_raw[col] = pd.to_numeric(energy_daily_raw[col], errors='coerce')

# 4. Preencher NaNs com 0 (assumindo que "sem dados" ou vazio significa "equipamento desligado/consumo 0")
energy_daily_raw.fillna(0, inplace=True)

# 5. Converter de Wh para kWh (dividir por 1000) para facilitar a interpretação
energy_daily_kwh = energy_daily_raw.copy()
for col in equipment_ids:
    energy_daily_kwh[col] = energy_daily_kwh[col] / 1000

print("\n--- Validação do Dataset 1 ---")
print("Dataset 'energy_daily_kwh' (kWh) após preparação inicial (5 primeiras linhas):")
print(energy_daily_kwh.head())

print("\nInformações sobre o Dataset (dtypes e não-nulos):")
print(energy_daily_kwh.info())

print("\nEstatísticas Descritivas do Consumo Diário em kWh (amostra dos equipamentos):")
print(energy_daily_kwh.drop(columns=['timestamp']).describe().T.head(5).round(2)) # Amostra para não sobrecarregar

print(f"\n--- Confirmação de Equipamentos e Período ---")
print(f"Número TOTAL de equipamentos monitorados: {len(equipment_ids)}")
print(f"Número de dias (linhas) no dataset: {energy_daily_kwh.shape[0]}")
print(f"Período de dados: De {energy_daily_kwh['timestamp'].min().date()} a {energy_daily_kwh['timestamp'].max().date()}")

Dataset './data/energy30days.csv' carregado com sucesso.

--- Validação do Dataset 1 ---
Dataset 'energy_daily_kwh' (kWh) após preparação inicial (5 primeiras linhas):
                  timestamp  SMV340200  SMV340201  SMV340202  SMV340203  \
0 2025-06-02 21:00:00-03:00  16.949087  17.226529  16.672130   0.000000   
1 2025-06-03 21:00:00-03:00  17.414107  17.701620  17.580698  18.553354   
2 2025-06-04 21:00:00-03:00  17.715928  17.884871  17.540541  13.287821   
3 2025-06-05 21:00:00-03:00  17.809011  17.764612  17.605126  18.773111   
4 2025-06-06 21:00:00-03:00  17.592313  14.592936  17.562198  18.738851   

   SMV340204  SMV340205  SMV340206  SMV340207  SMV340208  ...  SMV340232  \
0  23.209424  22.016507  20.331707  21.045300  24.745333  ...  19.748702   
1  23.447659  22.282449  21.035293  22.078178  25.605770  ...  19.686185   
2  22.644880  22.471000  20.898207  22.201412  25.754032  ...  19.483610   
3  22.805498  22.437053  21.234131  22.214814  25.908580  ...  19.435091   
4